In [11]:
from urllib.parse import urlparse
import requests
import datetime
import json

# Disable SSL warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

DEBUG = False
MAX_REQUEST_RETRIES = 5

class Prometheus:
    """docstring for Prometheus."""
    def __init__(self, url='', end_time=None, data_chunk='1h',stored_data='1h'):
        #self.headers = { 'Authorization': "bearer {}".format(token) }
        self.url = url
        self.prometheus_host = urlparse(self.url).netloc
        self._all_metrics = None
        self.data_chunk_size = data_chunk
        self.end_time = datetime.datetime.now()
        self.stored_data_range = stored_data
        self.DATA_CHUNK_SIZE_LIST = {
            '1m' : 60,
            '3m' : 180,
            '5m' : 300,
            '30m': 1800,
            '1h' : 3600,
            '3h' : 10800,
            '6h' : 21600,
            '12h': 43200,
            '1d' : 86400,
            '2d' : 172800}

    def all_metrics(self):
        '''
        Get the list of all the metrics that the prometheus host has
        '''
        if not self._all_metrics:
            #response = requests.get('{0}/api/v1/label/__name__/values'.format(self.url),
                                   # verify=False, # Disable ssl certificate verification temporarily
                                   # headers=self.headers)
            response = requests.get('{0}/api/v1/label/__name__/values'.format(self.url),
                                    verify=False)
            if DEBUG:
               
                print("URL => ", response.url)
            if response.status_code == 200:
                self._all_metrics = response.json()['data']
            else:
                raise Exception("HTTP Status Code {} {} ({})".format(
                    response.status_code,
                    requests.status_codes._codes[response.status_code][0],
                    response.content
                ))
            print(response)
        return self._all_metrics

    def get_metric(self, name, chunks=None, data_size=None):
        if chunks:
            if str(chunks) in self.DATA_CHUNK_SIZE_LIST:
                self.data_chunk_size = str(chunks)
                pass
            else:
                print("Invalid Chunk Size, using default value: {}".format(self.data_chunk_size))
            pass
        if data_size:
            if str(data_size) in self.DATA_CHUNK_SIZE_LIST:
                self.stored_data_range = str(data_size)
                pass
            else:
                print("Invalid Data Size, using default value: {}".format(self.stored_data_range))
            pass

        if not name in self.all_metrics():
            raise Exception("{} is not a valid metric".format(name))
        elif DEBUG:
            print("Metric is valid.")

        # num_chunks = 1
        num_chunks = int(self.DATA_CHUNK_SIZE_LIST[self.stored_data_range]/self.DATA_CHUNK_SIZE_LIST[self.data_chunk_size]) # Calculate the number of chunks using total data size and chunk size.
        metrics = self.get_metrics_from_prom(name, num_chunks)
        if metrics:
            return metrics


    def get_metrics_from_prom(self, name, chunks):
        if not name in self.all_metrics():
            raise Exception("{} is not a valid metric".format(name))

        # start = self.start_time.timestamp()
        end_timestamp = self.end_time.timestamp()
        chunk_size = self.DATA_CHUNK_SIZE_LIST[self.data_chunk_size]
        start = end_timestamp - self.DATA_CHUNK_SIZE_LIST[self.stored_data_range] + chunk_size
        data = []
        for i in range(chunks):
            # gc.collect() # Garbage collect to save Memory
            if DEBUG:
                print("Getting chunk: ", i)
                print("Start Time: ",datetime.datetime.fromtimestamp(start))

            tries = 0
            while tries < MAX_REQUEST_RETRIES:  # Retry code in case of errors
                response = requests.get('{0}/api/v1/query'.format(self.url),    # using the query API to get raw data
                                        params={'query': name+'['+self.data_chunk_size+']',
                                                'time': start
                                                },
                                        verify=False)
                if DEBUG:
                    print(response.url)
                    pass

                tries+=1
                if response.status_code == 200:
                    data += response.json()['data']['result']

                    if DEBUG:
                        # print("Size of recent chunk = ",getsizeof(data))
                        # print(data)
                        print(datetime.datetime.fromtimestamp(response.json()['data']['result'][0]['values'][0][0]))
                        print(datetime.datetime.fromtimestamp(response.json()['data']['result'][0]['values'][-1][0]))
                        pass

                    del response
                    tries = MAX_REQUEST_RETRIES
                elif response.status_code == 504:
                    if tries >= MAX_REQUEST_RETRIES:
                        self.connection_errors_count+=1
                        return False
                    else:
                        print("Retry Count: ",tries)
                        sleep(CONNECTION_RETRY_WAIT_TIME)    # Wait for a second before making a new request
                else:
                    if tries >= MAX_REQUEST_RETRIES:
                        self.connection_errors_count+=1
                        raise Exception("HTTP Status Code {} {} ({})".format(
                            response.status_code,
                            requests.status_codes._codes[response.status_code][0],
                            response.content
                        ))
                    else:
                        print("Retry Count: ",tries)
                        sleep(CONNECTION_RETRY_WAIT_TIME)

            start += chunk_size

        return(json.dumps(data))

    def get_current_metric_value(self, metric_name, label_config = None):
        data = []
        if label_config:
            label_list = [str(key+"="+ "'" + label_config[key]+ "'") for key in label_config]
            # print(label_list)
            query = metric_name + "{" + ",".join(label_list) + "}"
        else:
            query = metric_name
        response = requests.get('{0}/api/v1/query'.format(self.url),    # using the query API to get raw data
                                params={'query': query},#label_config},
                                verify=False, # Disable ssl certificate verification temporarily
                                headers=self.headers)
        data += response.json()['data']['result']
        return (json.dumps(data))
        pass
def get_df_from_json(metric, metric_dict_pd={}, data_window=5):
    '''
    Method to convert a json object of a Prometheus metric to a dictionary of shaped Pandas DataFrames

    The shape is dict[metric_metadata] = Pandas Object

    Pandas Object = timestamp, value
                    15737933, 1
                    .....

    This method can also be used to update an existing dictionary with new data
    '''
    current_time = datetime.now()
    earliest_data_time = current_time - timedelta(days = data_window)


    print("Pre-processing Data...........")
    for row in metric:
        # metric_dict[str(row['metric'])] = metric_dict.get(str(row['metric']),[]) + (row['values'])
        metric_metadata = str(SortedDict(row['metric']))[11:-1] # Sort the dictionary and then convert it to string so it can be hashed
        # print(metric_metadata)
        # print("Row Values: ",row['values'])
        if  metric_metadata not in metric_dict_pd:
            metric_dict_pd[metric_metadata] = pandas.DataFrame(row['values'], columns=['ds', 'y']).apply(pandas.to_numeric, args=({"errors":"coerce"}))
            metric_dict_pd[metric_metadata]['ds'] = pandas.to_datetime(metric_dict_pd[metric_metadata]['ds'], unit='s')
            pass
        else:
            temp_df = pandas.DataFrame(row['values'], columns=['ds', 'y']).apply(pandas.to_numeric, args=({"errors":"coerce"}))
            temp_df['ds'] = pandas.to_datetime(temp_df['ds'], unit='s')
            metric_dict_pd[metric_metadata] = metric_dict_pd[metric_metadata].append(temp_df, ignore_index=True)
            mask = (metric_dict_pd[metric_metadata]['ds'] > earliest_data_time)
            metric_dict_pd[metric_metadata] = metric_dict_pd[metric_metadata].loc[mask]
            pass
        metric_dict_pd[metric_metadata] = metric_dict_pd[metric_metadata].dropna()
        metric_dict_pd[metric_metadata] = metric_dict_pd[metric_metadata].drop_duplicates('ds').sort_values(by=['ds']).reset_index(drop = True)

        if len(metric_dict_pd[metric_metadata]) == 0:
            del metric_dict_pd[metric_metadata]
            pass
        pass

    return metric_dict_pd


def predict_metrics(pd_dict, prediction_range=1):
    '''
    This Function takes input a dictionary of Pandas DataFrames, trains the Prophet model for each dataframe and returns a dictionary of predictions.
    '''

    total_label_num = len(pd_dict)
    # LABEL_LIMIT = limit_labels
    PREDICT_DURATION = prediction_range

    current_label_num = 0
    limit_iterator_num = 0

    predictions_dict = {}

    for meta_data in pd_dict:
        try:
            current_label_num += 1
            limit_iterator_num += 1

            print("Training Label {}/{}".format(current_label_num,total_label_num))
            data = pd_dict[meta_data]

            print("----------------------------------\n")
            print(meta_data)
            print("Number of Data Points: {}".format(len(pd_dict[meta_data])))
            print("----------------------------------\n")

            data['ds'] = pandas.to_datetime(data['ds'], unit='s')

            train_frame = data

            # Prophet Modelling begins here
            m = Prophet(daily_seasonality = True, weekly_seasonality=True)

            print("Fitting the train_frame")
            m.fit(train_frame)

            future = m.make_future_dataframe(periods=int(PREDICT_DURATION))

            forecast = m.predict(future)

            # To Plot
            fig1 = m.plot(forecast)
            #
            fig2 = m.plot_components(forecast)
            forecast['timestamp'] = forecast['ds']
            forecast = forecast[['timestamp','yhat','yhat_lower','yhat_upper']]
            forecast = forecast.set_index('timestamp')

            # Store predictions in output dictionary
            predictions_dict[meta_data] = forecast

            # forecast.plot()
            # plt.legend()
            # plt.show()
        except ValueError as exception:
            if str(exception) == "ValueError: Dataframe has less than 2 non-NaN rows.":
                print("Too many NaN values........Skipping this label")
                limit_iterator_num -= 1
            else:
                raise exception
        pass

    return predictions_dict


In [12]:
url = "http://61.28.251.119:9090"

In [13]:
# get metric value 
prom = Prometheus(url, '1h', '1h')

In [14]:
metric = prom.get_metric('mem_used')

<Response [200]>


In [15]:
import json

In [16]:
## convert metric to json 
metric = json.loads(metric)

In [17]:
len(metric[0])

2

In [19]:
# convert json to data frame
import model
data_dict = {}
data_dict = get_df_from_json(metric, data_dict, data_window)

NameError: name 'data_window' is not defined